In [1]:
import cv2 
import os
import numpy as np 


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

2024-04-16 23:34:23.608636: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 23:34:23.608708: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 23:34:23.814168: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-16 23:34:24.187627: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-16 23:34:25.655327: W tensorflow/compiler/tf2

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

2024-02-10 19:32:50.627898: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-10 19:32:51.666147: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-10 19:32:51.666292: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
class L1Dist(Layer):
    def __init__(self,**kwargs):
        super().__init__()

    def call(self,input_embedding,validation_embedding):
        return tf.math.abs(input_embedding-validation_embedding)

In [4]:
model = tf.keras.models.load_model(
    'siamesemodel.h5',
    custom_objects={'L1Dist':L1Dist})

2024-02-10 19:32:51.747478: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-10 19:32:51.747656: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-10 19:32:51.747757: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [8]:
def preprocessing(file_path):

    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img,(100,100))
    img = img /255.0
    return img


In [6]:
def verify( model,detectthresh,verifthresh):

    results=[]


    for img in os.listdir('verification'):
        input_img = preprocessing('inputimage/input_image.jpg')
        validation_img = preprocessing(os.path.join('verification',img ))

        
        result = model.predict(list(np.expand_dims([input_img,validation_img],axis=1)))
        print(result)
        results.append(result)

    detection = np.sum(np.array(results)>detectthresh)

    verification = detection/len(os.listdir('verification'))

    verified = verification>verifthresh

    return results , verified

In [9]:
cap = cv2.VideoCapture(0)

while cap.isOpened():

    ret,frame = cap.read()
    frame=frame[170:170+250,190:190+250]
    
    cv2.imshow('verification',frame)

    if cv2.waitKey(1) & 0XFF == ord('v'):
        cv2.imwrite('inputimage/input_image.jpg',frame)

        results,verified = verify(model,0.5,0.5)
        print(verified)
        

    if cv2.waitKey(1) & 0XFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

2024-02-10 19:36:30.091052: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-02-10 19:36:30.700449: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 [==============================] - 3s 3s/step


2024-02-10 19:36:32.919405: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


[[0.9519857]]
1/1 [==============================] - 0s 19ms/step
[[1.]]
1/1 [==============================] - 0s 15ms/step
[[0.985446]]
1/1 [==============================] - 0s 16ms/step
[[0.6997154]]
1/1 [==============================] - 0s 18ms/step
[[1.]]
1/1 [==============================] - 0s 17ms/step
[[0.99999976]]
1/1 [==============================] - 0s 18ms/step
[[0.98089623]]
1/1 [==============================] - 0s 16ms/step
[[0.99999976]]
1/1 [==============================] - 0s 25ms/step
[[1.]]
1/1 [==============================] - 0s 18ms/step
[[0.918262]]
1/1 [==============================] - 0s 21ms/step
[[0.99999845]]
1/1 [==============================] - 0s 16ms/step
[[0.9843769]]
1/1 [==============================] - 0s 18ms/step
[[0.99999976]]
1/1 [==============================] - 0s 19ms/step
[[0.95302844]]
1/1 [==============================] - 0s 16ms/step
[[0.51138276]]
1/1 [==============================] - 0s 18ms/step
[[0.8497273]]
1/1 [=======